## Bilinear-Biquadratic Chain

The relevance of this Hamiltonian lies in its ability to capture a wide range of physical phenomena, such as phase transitions and the exotic states of matter.
$$
H=J\sum_i \cos\theta \left( \boldsymbol S_i\cdot \boldsymbol S_{i+1}\right) + \sin\theta\left(\boldsymbol S_i\cdot \boldsymbol S_{i+1}\right)^2
$$

The phases of the bilinear-biquadratic chain with respect to $\theta$. For reference the antiferromagnetic model is $\theta=0$.
![](images/bb_phases.png)

In [ ]:
using MPSKitModels, MPSKit, TensorKit, TensorOperations, Plots

include("hamiltonian.jl")
include("groundstate.jl")
include("plot_spectrum.jl")

using Logging
disable_logging(LogLevel(Info))

## Entanglement Spectra

Schmidt decomposition:
$$
\ket \Psi = \sum_{\alpha=1}^D\lambda_\alpha \ket \Phi^A_\alpha \otimes \ket \Phi^B_\alpha
$$
The Schmidt coefficients resemble entanglement. High coefficients mean more entanglement whilst $\lambda_\alpha=\{1, 0, 0, ...\}$ means we have a product state.

We can approximate the state by truncating the MPS when the Schmidt coefficients are sufficiently low.

#### Ferromagnetic spin=1:
- product state

In [ ]:
HFM = bilinear_biquadratic_hamiltonian(spin=1, θ=π)
Ψ_HFM = optimize_groundstate(HFM, bond=2, maxiter=500)
spectrum_HFM = entanglement_spectrum(Ψ_HFM)

plot_entanglement(spectrum_HFM)
hline!([1e-10], color=:black, label=nothing)
display(current())

![](images/HFM_spectrum.svg)

#### AntiFerromagnetic spin=1:
- in SPT phase

In [ ]:
HAFM = bilinear_biquadratic_hamiltonian(spin=1, θ=0)
Ψ_HAFM = optimize_groundstate(HAFM, bond=12, maxiter=500)
spectrum_HAFM = entanglement_spectrum(Ψ_HAFM)

plot_entanglement(spectrum_HAFM)
display(current())

![](images/HAFM_spectrum.svg)

#### AKLT:
- $\theta = \arctan(1/3)$
- exactly represented by $\textrm{bond}=2$

In [ ]:
AKLT = bilinear_biquadratic_hamiltonian(spin=1, θ=atan(1/3))
Ψ_AKLT = optimize_groundstate(AKLT, bond=6, maxiter=500)
spectrum_AKLT = entanglement_spectrum(Ψ_AKLT)

plot_entanglement(spectrum_AKLT)
display(current())

![](images/AKLT_spectrum.svg)

## SPT

Choose a state with a symmetry group $\mathrm G$.

The *Fundamental theorem of MPS* $\implies$ different equivalence classes.

Two states of the same 'class' can be connected via a path of gapped parent hamiltonians, thus no phase transitions $\implies$ class = phase.

If we travel from one class to another we must cross a phase transition if the symmetry is preserved $\implies$ symmetry protected.

The amount of equivalence classes is equal to the order of the second cohomology group $H^2(\mathrm G, \mathrm U_1)$.

Symmetry has to be represented by a **physical**¹ **linear rep.** but can be represented by a **virtual projective rep.**


#### AntiFerromagnet spin=1

$H^2(\mathrm{SO}_3, \mathrm U_1)=\mathbb Z_2\implies$ 2 phases:
- Trivial phase: **linear rep.**
- Haldane phase: **projective rep.**

The half integer spin reps. are projective reps. of $\mathrm{SO}_3$. The Haldane phase thus has half integer reps.

The right fixed point $\rho$ is invariant under our virtual symmetry:
$$
X_g\rho X_g^\dagger = \rho \implies \textrm{$\rho$ is proportional to }
$$
The degeneracy of the eigenvalues of $\rho$, which are the Schmidt coefficients, are determined by the irreps. of $X_g$.

## Degeneracies

These degeneracies are always even for our spin=1 $\mathrm{SO}_3$ state as the **virtual projective reps.** are the half integer reps. which have even dimension.

In [ ]:
plot_entanglement(spectrum_HAFM)
annotate!(2 + 1, spectrum_HAFM[2], text("spin=1/2", :red, 10))
annotate!(6 + 1, spectrum_HAFM[6], text("spin=3/2", :red, 10))
annotate!(8 + 1, spectrum_HAFM[8], text("spin=1/2", :red, 10))
display(current())

![](images/HAFM_spectrum_decomposition.svg)

## Convergence

If we optimize the HAFM for $\textrm{bond}=7$ we need more iterations to converge, whilst $\textrm{bond}=8$ gets fast convergence with less iterations.

The state is approximated by a direct sum of projective irreps on a virtual level and cutting one of these irreps in half makes for a harder approximation.

In [ ]:
Ψ = optimize_groundstate(HAFM, bond=7, maxiter=500) #around ~250 iterations
entanglement_spectrum(Ψ)

In [ ]:
Ψ = optimize_groundstate(HAFM, bond=8, maxiter=500) #around ~40 iterations
entanglement_spectrum(Ψ)

## Approaching AKLT

$\theta\to \arctan(1/3)\approx 0.3218$

For increasing $\theta$ we get an increase in the excitation energy of the magnon. The gap increases and we get overal decreased entanglement. 

Let's now plot the entanglement spectra for increasing $\theta$:


In [ ]:
θ_range = linrange(0, atan(1/3), 20; endpoint=false)

Ψ = bilinear_biquadratic_θ_range(θ_range, spin=1, bond=24, maxiter=500)
spectra_AKLT = entanglement_spectra(Ψ)

plot_spectra(θ_range, "θ", spectra_AKLT)
display(current())

![](images/AKLT_spectra_theta.svg)

We take the Schmidt value $\chi=3$ and see how it changes for each $\theta$:

In [ ]:
χ=3
scatter(θ_range, spectra_AKLT[:, χ], yscale=:log10, xlabel="θ", ylabel="Schmidt coefficient $(χ)", legend=false, color=:black)
ticks = logticks(correlations)
yticks!(ticks) 
ylims!((ticks[1], last(ticks)))
vline!([atan(1/3)], label=nothing)
display(current())

![](images/AKLT_chi=3.svg)

At $\theta = \arctan(1/3)$ (red line) we get the AKLT ground state which is exactly described by an MPS with $\textrm{bond}=2$, thus the other Schmidt coefficients go to zero.

## Approaching Nematic

The phase beyond $\frac{\pi}{4}$ is called the Nematic phase.

Let's now look at the correlation length $\xi$ for $\theta \in [0, \frac{\pi}{4}[$. 

We will plot the correlation length for $\theta$ approaching this Nematic phase.

In [ ]:
θ_range = linrange(0, π/4, 20; endpoint=false)
Ψ = bilinear_biquadratic_θ_range(θ_range, spin=1, bond=24, maxiter=500)
correlations = correlation_lengths(Ψ)

scatter(θ_range, correlations, yscale=:log10, xlabel="θ", ylabel="ξ", legend=false, color=:black)
ticks = logticks(correlations)
yticks!(ticks) 
ylims!((ticks[1], last(ticks)))
vline!([atan(1/3)], label=nothing, color=:red)
vline!([π/4], label=nothing, color=:red)
display(current())

![](images/bb_correlations_theta.svg)

If we now choose $\theta=\frac{\pi}{4}$ and increase the bond dimension we will see that the correlation length grows exponentially.  
This makes sense as the correlation length for this critical point (gapless) should be infinite.  
But since MPS are poor approximations for gapless states² we see this 'artifact' of ever increasing $\xi$ for increasing bond dimension.  

In [ ]:
bond_range = [12,24,30,48] # these converge quickly even at this critical point
H_Nematic = bilinear_biquadratic_hamiltonian(spin=1, J=1.0, θ=π/4)
correlations = correlation_bond_range(bond_range, H_Nematic, maxiter=1000)

plot_correlations(bond_range, "bond dimension", correlations)
xticks!(Int.(bond_range))

![](images/bb_correlations_bond.svg)

## Haldane $\to$ Trivial Phase

The relevant symmetries of the HAFM are:
- $\mathrm{SO}_3$ rotation group
- $\mathbb Z_2 \times \mathbb Z_2$ subgroup of $\mathrm{SO}_3$:  
The elements of $\mathbb Z_2$ can be represented as: the trivial element (no rotation) and the $\pi$ radians rotation.  
The 4 elements of $\mathbb Z_2 \times \mathbb Z_2$ can then be represented as:
1. Identity element
2. $\pi$ rotation about some axis
3. $\pi$ rotation about an orthogonal axis
4. a rotation about both axes
- $\mathbb Z_2^T$ time reversal:  
This flips the spins $\boldsymbol S \to - \boldsymbol S$ (Similar to angular momentum: $\boldsymbol L = \boldsymbol r \times \boldsymbol p \to \boldsymbol L = \boldsymbol r \times (-\boldsymbol p)$)
- $\mathbb Z_2^P$ parity:  
We can flip the spin chain around a site or around a bond. The HAFM is invariant under both.  
If however the translation invariance is broken these symmetries will also be broken.

As mentioned earlier if the symmetry (with non-trivial proj. rep.) is preserved along a path of gapped parent hamiltonians we must cross a phase transition if we wish to return to the trivial phase from the SPT phase.

We will now travel to the trivial phase by:
1. preserving some of the symmetry and hence crossing a phase transition 
2. by breaking all the symmetry and hence not crossing a phase transition

#### Preserving Symmetry

The hamiltonian with perturbation:
$$
H=\sum_i J \boldsymbol S_i\cdot \boldsymbol S_{i+1} + g \left(S_i^z\right)^2
$$
breaks all symmetries of the original HAFM except for the $\mathbb Z_2 \times \mathbb Z_2$ subgroup of $\mathrm{SO}_3$ and time reversal $\mathbb Z_{2}^T$. To see this:
- $\mathbb Z_2 \times \mathbb Z_2$:  
Any rotation of $\pi$ radians along the cartesian axes will transform $S^z$ to $\pm S^z$, hence $\left(S^z\right)^2$ is invariant.
- $\mathbb Z_2^T$:  
Flips the spin $S^z \to - S^z$ so again $\left(S^z\right)^2$ is invariant.
- $\mathbb Z_2^P$:  
Doesn't break translation invariance and is invariant under any flipping of the spin chain as this is just an one-site operator.

If we now make $g \to +\infty$ we should end up in a product state $\ket {000...}$ which is of course no longer in an SPT phase, hence we must cross a phase transition.

Let's look at the spectrum for $g=0.1$ and $g=1.5$:

In [ ]:
Ψ = optimize_groundstate(HAFM_zz(spin=1, J=1.0, g=.1), bond=24, maxiter=500)
spectrum = entanglement_spectrum(Ψ)

plot_entanglement(spectrum)
display(current())

![](images/zz_spectrum_g=0.1.svg)

We see that some of the degeneracies have split into smaller groups. This can be understood as the breaking of the $\mathrm{SO}_3 \to \mathbb Z_2 \times \mathbb Z_2$ subgroup.

In [ ]:
Ψ = optimize_groundstate(HAFM_zz(spin=1, J=1.0, g=1.5), bond=24, maxiter=500)
spectrum = entanglement_spectrum(Ψ)

plot_entanglement(spectrum)
display(current())

![](images/zz_spectrum_g=1.5.svg)

Most of the degeneracy seems to have dissapeard for $g=1.5$. This might suggest we have crossed a phase transition.

Now let's look at the correlation length for varying $g$ to see if we can find a phase transition:

In [ ]:
g_range = linrange(0, 1.5, 10)
Ψ = perturbation_range(HAFM_zz, g_range, J=1.0, spin=1, bond=24, maxiter=500)
correlations = correlation_lengths(Ψ)

plot_correlations(g_range, "g", correlations)
display(current())

![](images/zz_correlations_g_2.svg)

There is a peak at $g=1$, this peak again grows for increasing bond dimension. As expected there is a phase transition.

In [ ]:
bond_range = [12,24,48,64] # these converge quickly even for this perturbed hamiltonian
correlations = correlation_bond_range(bond_range, HAFM_zz(g=1.0), maxiter=1000)

plot_correlations(bond_range, "bond dimension", correlations)
xticks!(Int.(bond_range))
display(current())


![](images/zz_correlations_bond.svg)

#### Breaking all Symmetry

The hamiltonian with *staggered* perturbation:
$$
H=\sum_i J \boldsymbol S_i\cdot \boldsymbol S_{i+1} + (-1)^i g S_i^z
$$
breaks all symmetries of the original HAFM.
- $\mathbb Z_2 \times \mathbb Z_2$:  
is broken as rotating around the $x$ or $y$ axis flips the sign of $S^z$ which means it is already no longer invariant under $\mathbb Z_2$.
- $\mathbb Z_2^T$:  
flips the spins and hence also $S^z$.
- $\mathbb Z_2^P$:  
the staggered perturbation breaks the original translation invariance and thus breaks both parity symmetries.

Lets see what the spectrum looks like for a small $g$:

In [ ]:
Ψ = optimize_groundstate(HAFM_staggered(spin=1, J=1.0, g=0.1), bond=12, maxiter=500)
spectrum = entanglement_spectrum(Ψ)
plot_entanglement(spectrum)
display(current())

![](images/staggered_spectrum_g%3D0.1.svg)

We see that the degeneracy has completely dissapeared. The symmetry is fully broken. We can now travel to a product state without crossing a phase transition.

Lets also look at the spectra for $g\in[0,10]$ to see how it changes.

In [ ]:
g_range = linrange(0, 10, 20)
Ψ_staggered = perturbation_range(HAFM_staggered, g_range, J=1.0, spin=1, bond=24, maxiter=500)
spectra = entanglement_spectra(Ψ_staggered)

plot_spectra(g_range, "g", spectra)
display(current())

![](images/staggered_spectra_g.svg)

We see that the Schmidt coefficients go to zero as $g$ increases, it is going to the product state $\ket{+-+-...}$.

Lets look at the correlation length to see if there is indeed no phase transition:

In [ ]:
g_range = linrange(0, 10, 20)
Ψ = perturbation_range(HAFM_staggered, g_range, J=1.0, spin=1, bond=24, maxiter=500)
correlations = correlation_lengths(Ψ)

plot_correlations(g_range, "g", correlations)
display(current())

![](images/staggered_correlations_g.svg)

There is no divergence in the correlation length as we move to a product state and thus no phase transition as expected.

Notes:
1. It has been shown that the Haldane gap is only invariant under physical linear representations.  
This means that the physical symmetry must be represented by a linear representation. It can still however be represented projectively on a virtual level.
2. gapped H have exponential $\xi$ decay whilst gapless H have power law decay  
$\implies$ MPS, which also have exponential decay, are good to describe gapped states.